In [6]:
from langchain_community.tools import ArxivQueryRun ,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [8]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="use this tool to search for research papers on arxiv")
arxiv.invoke("What are the latest research papers on deep learning?")

ImportError: Could not import arxiv python package. Please install it with `pip install arxiv`.

In [ ]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki,description="use this tool to search for information on wikipedia")


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")


In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults()
tavily.invoke("What are the latest research papers on deep learning?")

C:\Users\Maviya Shaikh\AppData\Local\Temp\ipykernel_20264\2715124348.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults()


[{'title': 'Advancements in Deep Learning: A Comprehensive Study of the ...',
  'url': 'https://www.academia.edu/144859758/Advancements_in_Deep_Learning_A_Comprehensive_Study_of_the_Latest_Trends_and_Techniques_in_Machine_Learning',
  'content': 'This paper provides a comprehensive study of the latest trends and techniques in deep learning, a rapidly evolving field of machine learning. The paper begins by introducing the background of machine learning and the purpose of the study. Next, it provides an overview of deep learning, including its definition, history, key concepts, and techniques. The paper then examines the advancements in neural network architectures, including Convolutional Neural Networks (CNNs), Recurrent Neural Networks (RNNs), and Generative Adversarial Networks (GANs). The paper also explores emerging applications of deep learning in computer vision, natural language processing, and reinforcement learning. The paper concludes by discussing the challenges and limitati

In [12]:
tools=[arxiv,wiki,tavily]

NameError: name 'arxiv' is not defined

In [13]:
from langchain_groq import ChatGroq
llm=ChatGroq(model="qwen-2.5-32b")

In [ ]:
llm_with_tools=llm.bind_tools(tools=tools)

# Workflow

In [4]:
from typing_extensions import Annotated ,  TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

In [5]:
class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    

In [ ]:
# all graph libraries
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode ,tools_condition
from IPython.display import display ,Image


In [ ]:
def tool_calling_llm(state:State):
    return {"messages": llm_with_tools.invoke(state["messages"])}


# building the graph
builder=StateGraph(State)
builder.add_node("tool_calling",tool_calling_llm)
builder.add_node("tools",ToolNode(tools=tools))


#edges
builder.add_edge(START,"tool_calling")
builder.add_conditional_edges(
    "tool_calling",
                              
                 # if the llm calls a tool, then we go to the tools node
                 # if the llm does not call a tool, then we end the graph execution             
                              tools_condition,

)
builder.add_edge("tools","tool_calling")
builder.add_edge("tool_calling",END,condition=lambda state: not tools_condition(state))



NameError: name 'tools' is not defined

In [ ]:
# react architecture 
buil
